# Requirements

In [1]:
import pandas as pd
import numpy as np
import json

import language_tool_python

ModuleNotFoundError: No module named 'language_tool_python'

# Clean data of grammar mistakes

In [2]:
# load data
data = pd.read_csv('../raw_data/train.csv')

texts = data.loc[:, "full_text"].values
scores = data.loc[:, "phraseology"].values

In [ ]:
# clear grammar errors - takes time
tool = language_tool_python.LanguageTool('en-US')

clear_texts = texts.copy()
for i in range(texts.shape[0]):
    clear_texts[i] = tool.correct(texts[i])

In [40]:
# remove '.' and ',' symbols, cast to lowercase, split into words

data = []
for i in range(clear_texts.shape[0]):
    # data.extend(np.array([clear_texts[i].replace('.', '').replace(',', '').lower().split()]))
    data += [clear_texts[i].replace('.', '').replace(',', '').lower().split()]

data = np.array(data) # data is numpy.ndarray of lists

/Users/pavelpopov/.pyenv/versions/3.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [34]:
# save clean data
np.savez(
    "../preprocessed_data/phraseology.npz",
    texts=texts,
    clear_texts=clear_texts,
    data=data,
    scores=scores,
)

In [36]:
# load data
np_data = np.load("../preprocessed_data/phraseology.npz", allow_pickle=True)
data = np_data['data']
scores = np_data['scores']

# Tokenize data

In [2]:
# load data
np_data = np.load("../preprocessed_data/phraseology.npz", allow_pickle=True)
texts = np_data['clear_texts']
scores = np_data['scores']

In [34]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

encoded_data = tokenizer.batch_encode_plus(
    texts, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    # padding='max_length',
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='np'
)


In [37]:
print(encoded_data.keys())

print(type(encoded_data["input_ids"]))
print(encoded_data["input_ids"].shape)
print(encoded_data["input_ids"].shape)
# print(encoded_data["input_ids"][0])

print(type(encoded_data["token_type_ids"]))
print(encoded_data["token_type_ids"].shape)
# print(encoded_data["token_type_ids"][0])

print(type(encoded_data["attention_mask"]))
print(encoded_data["attention_mask"].shape)
# print(encoded_data["attention_mask"][0])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
<class 'numpy.ndarray'>
(3911,)
(3911,)
<class 'numpy.ndarray'>
(3911,)
<class 'numpy.ndarray'>
(3911,)


In [33]:
# save tokenized data
np.savez(
    "../preprocessed_data/tokenized.npz",
    input_ids=encoded_data["input_ids"],
    token_type_ids=encoded_data["token_type_ids"],
    attention_mask=encoded_data["attention_mask"],
    scores=scores,
)